In [1]:
!git clone https://github.com/RUCAIBox/HaluEval.git

Cloning into 'HaluEval'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 166 (delta 37), reused 33 (delta 9), pack-reused 70
Receiving objects: 100% (166/166), 37.92 MiB | 8.10 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Updating files: 100% (38/38), done.


In [1]:
!pip install transformers[torch]

In [2]:
import pickle
import torch
# import graphvite as gv
#import dataset
import nltk
import re
import spacy
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
!pip install transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

In [5]:
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login

notebook_login()

In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00


In [7]:
!wget -O processed_data.pkl https://www.dropbox.com/scl/fi/vp5c9hcyqnaottfre5yas/processed_data.pkl?rlkey=i0gko0chfwyj5fn4rhsx9o3ve&dl=0
!wget -O neighbours.pkl https://www.dropbox.com/scl/fi/lw1d9mlr7laf7wo5ladvu/neighbours-Zack.pkl?rlkey=s81bm5oxh8oy0hmziq8rbxh84&dl=0

--2023-12-07 00:14:56--  https://www.dropbox.com/scl/fi/vp5c9hcyqnaottfre5yas/processed_data.pkl?rlkey=i0gko0chfwyj5fn4rhsx9o3ve
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdbad543d681006aa1969a4211f.dl.dropboxusercontent.com/cd/0/inline/CI5Y4g9egomJqcK35KKBKhrcq7S0szDOIf4VzKEOoDziiTidpy-7FWUKBFlsr_Mc55kMLGg2B3SZyz1IwF8CtusF142Ii_ElukaR3IGsuDQRIqHM5O250WhFexyjs5Zsd_yt3gPoib-fGv8EQoipGZ6o/file# [following]
--2023-12-07 00:14:56--  https://ucdbad543d681006aa1969a4211f.dl.dropboxusercontent.com/cd/0/inline/CI5Y4g9egomJqcK35KKBKhrcq7S0szDOIf4VzKEOoDziiTidpy-7FWUKBFlsr_Mc55kMLGg2B3SZyz1IwF8CtusF142Ii_ElukaR3IGsuDQRIqHM5O250WhFexyjs5Zsd_yt3gPoib-fGv8EQoipGZ6o/file
Resolving ucdbad543d681006aa1969a4211f.dl.dropboxusercontent.com (ucdbad543d681006aa1969a4211f.dl.dropboxusercontent.com)... 162.125.8.15, 

In [19]:
!wget -O simple_wikidata5m.pkl https://udemontreal-my.sharepoint.com/:u:/g/personal/zhaocheng_zhu_umontreal_ca/EVcJpJAzkThPu1vjgJLohscBgwtPajhTZvCCd8nEg1GiwA?download=1
# !wget -O entity.txt.gz https://www.dropbox.com/s/bgmgvk8brjwpc9w/entity.txt.gz?dl=1
!wget -O neighbour_embs.pkl https://www.dropbox.com/scl/fi/0ts9lajzorxfbyse3ogj5/neighbours-Santa-Claus.pkl?rlkey=6jq6ezx65kp7tqp8kekf6tvf1&dl=0

--2023-12-07 00:19:28--  https://udemontreal-my.sharepoint.com/:u:/g/personal/zhaocheng_zhu_umontreal_ca/EVcJpJAzkThPu1vjgJLohscBgwtPajhTZvCCd8nEg1GiwA?download=1
Resolving udemontreal-my.sharepoint.com (udemontreal-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to udemontreal-my.sharepoint.com (udemontreal-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/zhaocheng_zhu_umontreal_ca/Documents/model/simple_wikidata5m.pkl?ga=1 [following]
--2023-12-07 00:19:29--  https://udemontreal-my.sharepoint.com/personal/zhaocheng_zhu_umontreal_ca/Documents/model/simple_wikidata5m.pkl?ga=1
Reusing existing connection to udemontreal-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 10019193069 (9.3G) [application/octet-stream]
Saving to: ‘simple_wikidata5m.pkl’

simple_wikidata5m.p 100%[===================>]   9.33G  86.2MB/s    in 2m 52s  

2023-12-07 00:22:21 (55

In [9]:
processed_file = open("processed_data.pkl", "rb")
graph_data = pickle.load(processed_file)
neighbours_file = open("neighbours.pkl", "rb")
neighbours = pickle.load(neighbours_file)
neighbour_embs_file = open("neighbour_embs.pkl", "rb")
neighbour_embs = pickle.load(neighbour_embs_file)

In [ ]:
file = open("simple_wikidata5m.pkl", "rb")
model = pickle.load(file)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings
alias2entity = dataset.wikidata5m.load_alias("./entity.txt.gz")

In [26]:
entity2id = graph_data.graph.entity2id
relation2id = graph_data.graph.relation2id
entity_embeddings = graph_data.solver.entity_embeddings
relation_embeddings = graph_data.solver.relation_embeddings

AttributeError: ignored

In [4]:
import json

# Opening JSON file
data_list = []
with open('HaluEval/data/qa_data.json') as json_file:
    for jsonObj in json_file:
        data_dict = json.loads(jsonObj)
        data_list.append(data_dict)



In [5]:
connection_dict = {obj["question"].strip().lower() : i for i, obj in enumerate(data_list)}

In [6]:
len(connection_dict)

10000

In [7]:
from datasets import load_dataset

def get_data(data_path):

  ds = load_dataset("json", data_files=data_path, split="train")
  ds = ds.train_test_split(test_size=0.2)
  return ds

#print(os.listdir())

ds = get_data("HaluEval/data/qa_data.json")
print(ds)

DatasetDict({
    train: Dataset({
        features: ['knowledge', 'question', 'right_answer', 'hallucinated_answer'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['knowledge', 'question', 'right_answer', 'hallucinated_answer'],
        num_rows: 2000
    })
})


In [10]:
print(neighbour_embs[0]["portland, oregon"]["embedding"].shape)
emb_average = torch.empty((512,), dtype=torch.int64)

for key in neighbour_embs[1683]:
    emb_average += neighbour_embs[1683][key]["embedding"] / len(neighbour_embs[1683])

print(emb_average, emb_average.shape)

(512,)
tensor([ 2.4049e+01,  1.3428e+14,  2.1475e+10,  1.0454e+00,  1.9274e+00,
         4.7245e+10,  2.3963e+01,  1.3428e+14,  1.0860e+00,  2.1475e+10,
         6.0130e+10,  8.1604e+10,  2.2853e+18,  1.0019e+14,  1.3429e+14,
         8.5899e+10,  6.0130e+10,  1.0737e+11,  9.3304e-01,  1.0019e+14,
         1.0019e+14,  1.0420e+00,  1.0019e+14,  1.0019e+14,  1.9965e+01,
         1.3428e+14,  8.0525e-02,  1.0019e+14,  1.1596e+11,  1.2885e+11,
        -3.5580e+18,  1.3428e+14, -3.7569e-02,  1.0018e+14,  1.1167e+11,
         1.3744e+11,  9.6986e-01,  1.0019e+14,  1.0019e+14,  1.0018e+14,
         1.3428e+14,  2.3622e+11,  1.3428e+14,  1.7180e+11,  1.9327e+11,
         1.3428e+14,  1.0019e+14,  1.3429e+14,  2.5770e+10,  1.7180e+11,
         2.1904e+11,  1.3429e+14,  1.0019e+14,  1.4603e+11,  2.1904e+11,
         1.0018e+14,  1.0019e+14,  1.0019e+14,  1.0019e+14,  6.0130e+10,
         2.2334e+11,  6.0315e+00,  1.0019e+14,  1.0019e+14,  4.3087e-02,
         1.0019e+14,  1.1340e+03,  1.0019e+1

In [26]:
import torch

def preprocess_function(examples):
  questions = [q.strip().lower() for q in examples["question"]]
  inputs = tokenizer(
      questions,
      [c.lower() for c in examples["knowledge"]],
      max_length=384,
      truncation="only_second",
      return_offsets_mapping=True,
      padding="max_length",
  )
  offset_mapping = inputs.pop("offset_mapping")
  answers = examples["right_answer"]
  start_positions = []
  end_positions = []
  graph_embeddings = []
  avg_neighbours_embeddings = []

  for i, offset in enumerate(offset_mapping):
      answer = answers[i]
      start_char = examples["knowledge"][i].find(answer)
      end_char = start_char + len(answer)
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label it (0, 0)
      if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

      corr_index = connection_dict[questions[i]]
      graph_index = graph_data[corr_index]
      embeddings = torch.zeros(384, 512)
      neighbours_list_index = corr_index
      emb_average = torch.zeros((512,), dtype=torch.int64)
      token_list = inputs.input_ids[i]
      for ent, embedding in graph_index["knowledge"].items():
          ent_token_list = [tokenizer(ent).input_ids[1:-1], tokenizer(" " + ent).input_ids[1:-1]]

          for i in range(len(token_list) - len(list(ent_token_list)[0])):
            if token_list[i:i+len(list(ent_token_list)[0])] in ent_token_list:
              embeddings[i:i+len(list(ent_token_list)[0]), :] = torch.broadcast_to(torch.from_numpy(embedding), (len(list(ent_token_list)[0]), 512))
      for ent, embedding in graph_index["question"].items():
          ent_token_list = [tokenizer(ent).input_ids[1:-1], tokenizer(" " + ent).input_ids[1:-1]]

          for i in range(len(token_list) - len(list(ent_token_list)[0])):
            if token_list[i:i+len(list(ent_token_list)[0])] in ent_token_list:
              embeddings[i:i+len(list(ent_token_list)[0]), :] = torch.broadcast_to(torch.from_numpy(embedding), (len(list(ent_token_list)[0]), 512))
      graph_embeddings.append(embeddings)
      for key in neighbour_embs[neighbours_list_index]:
          emb_average += neighbour_embs[neighbours_list_index][key]["embedding"] / len(neighbour_embs[neighbours_list_index])
      avg_neighbours_embeddings.append(emb_average)


  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  inputs["graph_embeddings"] = graph_embeddings
  inputs["avg_neighbour_embeddings"] = avg_neighbours_embeddings

  return inputs

In [27]:
ds["train"][1]

{'knowledge': ' It is a subsidiary of WesCEF, which in turn is part of the industrials division of the Wesfarmers conglomerate.Wesfarmers Limited is an Australian conglomerate, headquartered in Perth, Western Australia, with interests predominantly in Australian and New Zealand retail, chemicals, fertilisers, coal mining and industrial and safety products.',
 'question': 'In which city is the headquarters of the parent company of WesCEF ?',
 'right_answer': 'Perth',
 'hallucinated_answer': 'The headquarters is in Sydney.'}

In [28]:
print(ds["train"][1]["question"].strip())
corr_index = connection_dict[ds["train"][1]["question"].strip().lower()]
print(corr_index)
#print(graph_data[corr_index])
graph_index = graph_data[corr_index]
#for ent, embedding in graph_index["knowledge"].items():
#    graph_embeddings.append(embedding)
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

for k, v in graph_index["question"].items():
  print("{}, {}, {}".format(k, tokenizer(k).input_ids[1:-1], tokenizer(" " + k).input_ids[1:-1]))
for k, v in graph_index["knowledge"].items():
  print("{}, {}, {}".format(k, tokenizer(k).input_ids[1:-1], tokenizer(" " + k).input_ids[1:-1]))
print(tokenizer("Teen Wolf"))
print(str(tokenizer.decode([664, 2143, 417])) + "," + str(tokenizer.decode([39556])))

In which city is the headquarters of the parent company of WesCEF ?
8571
wescef, [605, 293, 1755, 506], [885, 293, 1755, 506]
wescef, [605, 293, 1755, 506], [885, 293, 1755, 506]
wesfarmers, [605, 293, 19779, 268], [885, 293, 19779, 268]
wesfarmers limited, [605, 293, 19779, 268, 1804], [885, 293, 19779, 268, 1804]
australian, [102, 4193, 7085, 811], [28410, 7085, 811]
perth, [1741, 212], [228, 212]
western australia, [16507, 28410, 7085, 493], [4669, 28410, 7085, 493]
new zealand, [4651, 33103, 463], [92, 33103, 463]
{'input_ids': [0, 39556, 7602, 2], 'attention_mask': [1, 1, 1, 1]}
 young boyd,Teen


In [30]:
example = {
    "knowledge": [ds["train"][1]["knowledge"]],
    "question": [ds["train"][1]["question"]],
    "right_answer": [ds["train"][1]["right_answer"]],
    "hallucinated_answer": [ds["train"][1]["hallucinated_answer"]]
}
#print(clean_text(example))
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = preprocess_function(example)
print(inputs)
print(len(inputs["avg_neighbour_embeddings"]))
print(tokenizer.decode(inputs["input_ids"][0]))

1 512
{'input_ids': [[0, 179, 61, 343, 16, 5, 4351, 9, 5, 4095, 138, 9, 885, 293, 1755, 506, 17487, 2, 2, 24, 16, 10, 8540, 9, 885, 293, 1755, 506, 6, 61, 11, 1004, 16, 233, 9, 5, 25389, 1069, 1536, 2757, 9, 5, 885, 293, 19779, 268, 15814, 4, 605, 293, 19779, 268, 1804, 16, 41, 28410, 7085, 811, 15814, 6, 15830, 11, 228, 212, 6, 4669, 28410, 7085, 493, 6, 19, 3168, 15351, 11, 28410, 7085, 811, 8, 92, 33103, 463, 2304, 6, 8321, 6, 18928, 13844, 6, 4051, 4481, 8, 2683, 8, 1078, 785, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [31]:
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_ds = ds.map(preprocess_function, batched=True, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

1000 512
1000 512
1000 512
1000 512
1000 512
1000 512
1000 512
1000 512


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

1000 512
1000 512


In [32]:
#print(tokenized_ds["train"][0])
print(tokenized_ds["train"][0]["graph_embeddings"])
print(len(tokenized_ds["train"][0]["graph_embeddings"]))
print(len(tokenized_ds["train"][0]["graph_embeddings"][0]))
# print(tokenized_ds["train"]["end_positions"][0:10])

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [33]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [54]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import QuestionAnsweringModelOutput

class GraphEnrichedRoberta(nn.Module):
  def __init__(self, model_name):
    super().__init__()
    self.llm_embedding_size = 768
    self.graph_embedding_size = 512
    self.llm = AutoModelForQuestionAnswering.from_pretrained(model_name)
    self.fc = nn.Linear(self.graph_embedding_size + self.llm_embedding_size, self.llm_embedding_size)
    self.fc2 = nn.Linear(self.llm_embedding_size + self.graph_embedding_size, self.llm_embedding_size)

    for param in self.llm.roberta.encoder.parameters():
      param.requires_grad = False

  def forward(self, input_ids, attention_mask, start_positions, end_positions, graph_embeddings, avg_neighbour_embeddings):
     outputs = self.llm.roberta(input_ids, attention_mask)
     sequence_outputs = outputs.last_hidden_state

     enriched_encodings = self.fc(torch.cat((sequence_outputs, graph_embeddings), 2))
     enriched_encodings = F.relu(enriched_encodings)
     avg_neighbour_embeddings = avg_neighbour_embeddings[:, None, :]
     avg_neighbour_embeddings = (avg_neighbour_embeddings.repeat(1, 384, 1))
     final_encodings = self.fc2(torch.cat((enriched_encodings, avg_neighbour_embeddings), 2))
     final_encodings = F.relu(final_encodings)

     logits = self.llm.qa_outputs(final_encodings)
     start_logits, end_logits = logits.split(1, dim=-1)
     start_logits = start_logits.squeeze(-1).contiguous()
     end_logits = end_logits.squeeze(-1).contiguous()

     total_loss = None
     if start_positions is not None and end_positions is not None:
          # If we are on multi-GPU, split add a dimension
         if len(start_positions.size()) > 1:
             start_positions = start_positions.squeeze(-1)
         if len(end_positions.size()) > 1:
             end_positions = end_positions.squeeze(-1)
          # sometimes the start/end positions are outside our model inputs, we ignore these terms
         ignored_index = start_logits.size(1)
         start_positions = start_positions.clamp(0, ignored_index)
         end_positions = end_positions.clamp(0, ignored_index)

         loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
         start_loss = loss_fct(start_logits, start_positions)
         end_loss = loss_fct(end_logits, end_positions)
         total_loss = (start_loss + end_loss) / 2

     return QuestionAnsweringModelOutput(
         loss=total_loss,
         start_logits=start_logits,
         end_logits=end_logits,
         hidden_states=outputs.hidden_states,
         attentions=outputs.attentions,
     )

In [55]:
model = GraphEnrichedRoberta(model_name)

In [56]:
with torch.no_grad():
  input_ids_test = torch.tensor(tokenized_ds["train"][0]["input_ids"]).unsqueeze(0)
  attn_mask_test = torch.tensor(tokenized_ds["train"][0]["attention_mask"]).unsqueeze(0)
  start_positions_test = torch.tensor(tokenized_ds["train"][0]["start_positions"]).unsqueeze(0)
  end_positions_test = torch.tensor(tokenized_ds["train"][0]["end_positions"]).unsqueeze(0)
  graph_embeddings_test = torch.tensor(tokenized_ds["train"][0]["graph_embeddings"]).unsqueeze(0)
  avg_neighbour_embeddings_test = torch.tensor(tokenized_ds["train"][0]["avg_neighbour_embeddings"]).unsqueeze(0)
  print(model(input_ids_test, attn_mask_test, start_positions_test, end_positions_test, graph_embeddings_test, avg_neighbour_embeddings_test))

QuestionAnsweringModelOutput(loss=tensor(5.9157), start_logits=tensor([[ 0.0434, -0.0307, -0.0352, -0.0536, -0.0571, -0.0441, -0.0478, -0.0667,
         -0.0472, -0.0708, -0.0423, -0.0272, -0.0532, -0.0634, -0.0203, -0.0596,
         -0.0672, -0.0567, -0.0390, -0.0351, -0.0369, -0.0184, -0.0212, -0.0227,
         -0.0175, -0.0467, -0.0397, -0.0387, -0.0808, -0.0428, -0.0614, -0.0182,
         -0.0132, -0.0347, -0.0444, -0.0494, -0.0273, -0.0291, -0.0061, -0.0304,
          0.0030, -0.0505, -0.0269, -0.0269, -0.0372, -0.0337, -0.0284, -0.0323,
         -0.0480, -0.0580, -0.0690, -0.0277, -0.0283, -0.0234, -0.0524, -0.0512,
         -0.0390, -0.0424, -0.0554, -0.0338, -0.0651, -0.0697, -0.0360, -0.0294,
         -0.0445, -0.0543, -0.0421, -0.0498, -0.0419, -0.0650, -0.0439, -0.0424,
         -0.0536, -0.0529, -0.0806, -0.0378, -0.0572, -0.0418, -0.0418, -0.0418,
         -0.0418, -0.0418, -0.0418, -0.0418, -0.0418, -0.0418, -0.0418, -0.0418,
         -0.0418, -0.0418, -0.0418, -0.0418, -

In [57]:
training_args = TrainingArguments(
    output_dir="roberta_qa_model_with_graph",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

NameError: ignored

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.072100,No log
2,1.586100,No log


Epoch,Training Loss,Validation Loss
1,2.072100,No log
2,1.586100,No log
3,1.510300,No log


TrainOutput(global_step=1500, training_loss=1.7228113606770834, metrics={'train_runtime': 10196.5166, 'train_samples_per_second': 2.354, 'train_steps_per_second': 0.147, 'total_flos': 0.0, 'train_loss': 1.7228113606770834, 'epoch': 3.0})

In [ ]:
torch.save(model.state_dict(), 'graph_model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('graph_model_weights.pth'))

<All keys matched successfully>

In [ ]:
model

GraphEnrichedRoberta(
  (llm): RobertaForQuestionAnswering(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): 

In [ ]:
#tokenized_ds["test"]

predictions = trainer.predict(tokenized_ds["test"].remove_columns(["graph_embeddings"]))

In [ ]:
import numpy as np

for i in range(len(predictions.predictions)):
  print(predictions.predictions[i].shape)

print(predictions.predictions[0])

print(np.array(tokenized_ds["test"]["start_positions"]).shape)


(125,)
(2000, 384)
(2000, 384)
[1.0641084  1.6659701  1.9759023  1.2464397  1.970095   1.2182294
 1.7953851  1.3778882  1.089099   2.3120296  1.0138576  1.7308497
 1.5308025  1.2966044  1.4876668  0.62224865 2.2005458  1.19306
 1.9669992  1.8536528  2.293821   1.792011   0.7256372  1.629729
 2.1631832  1.4701074  1.4578855  1.6019956  1.2260633  1.738751
 2.4187698  1.3923588  1.9579014  1.3781996  2.456057   1.0412458
 1.7440045  1.8541543  2.8645754  0.93838847 1.7694396  1.6841547
 1.9321573  1.4411316  1.38648    1.194206   0.81366175 1.0065814
 0.8059305  1.9596742  0.79150534 1.9571393  0.9703902  1.6299062
 1.890945   1.146168   1.0196122  1.1374956  2.0560458  1.7159839
 1.968475   1.3967826  1.6296978  1.5666738  1.2158916  1.7590964
 2.4377935  1.9311898  1.9091766  2.4590755  1.9080064  1.0684565
 1.7132566  2.3912418  1.6143143  1.9749919  1.1747274  1.9486256
 1.5132129  1.6976893  1.2747147  1.1367294  2.944383   1.1048306
 1.9857776  1.3243711  1.0923741  1.5487192  1.53

In [ ]:
avg_loss = np.mean(predictions.predictions[0])
avg_loss

1.6121981

In [ ]:
start_logits = predictions.predictions[1]
end_logits = predictions.predictions[2]

start_positions = np.array(tokenized_ds["test"]["start_positions"])
end_positions = np.array(tokenized_ds["test"]["end_positions"])

In [ ]:
start_positions_pred = np.argmax(start_logits, axis=1)
end_positions_pred = np.argmax(end_logits, axis=1)

In [ ]:
print(start_positions_pred)
print(start_positions)
print(end_positions_pred)
print(end_positions)
print(np.sum(start_positions_pred == start_positions))
overlap = np.maximum(0.0, np.minimum(end_positions_pred, end_positions) - np.maximum(start_positions_pred, start_positions) + 1)
print(overlap)
val1 = np.abs(1.0 * overlap / (end_positions - start_positions + 1))
val2 = np.abs(1.0 * overlap / (end_positions_pred - start_positions_pred + 1))
print(np.min(val1))
print(np.max(val1))
print(np.min(val2))
print(np.max(val2))

[ 39  55  97 ... 127  51  24]
[ 39  55  97 ... 127  51  24]
[ 42  57 102 ... 130  56  29]
[ 42  57 102 ... 130  56  29]
1165
[4. 3. 6. ... 4. 6. 6.]
0.0
1.0
0.0
1.0


In [ ]:
def exact_match(start_logits, end_logits, start_positions, end_positions):
  start_positions_pred = np.argmax(start_logits, axis=1)
  end_positions_pred = np.argmax(end_logits, axis=1)
  same_start = start_positions_pred == start_positions
  same_end = end_positions_pred == end_positions
  return np.sum(np.logical_and(same_start, same_end)) / start_positions.shape[0]

In [ ]:
exact_match(start_logits, end_logits, start_positions, end_positions)

0.505

In [ ]:
def f1_score(start_logits, end_logits, start_positions, end_positions):
  start_positions_pred = np.argmax(start_logits, axis=1)
  end_positions_pred = np.argmax(end_logits, axis=1)
  overlap = np.maximum(0, np.minimum(end_positions_pred, end_positions) - np.maximum(start_positions_pred, start_positions) + 1)
  recall = np.abs(1.0 * overlap / (end_positions - start_positions + 1))
  precision = np.abs(1.0 * overlap / (end_positions_pred - start_positions_pred + 1))
  f1 = (2 * precision * recall) / (precision + recall + 0.00000001)
  return np.mean(f1)

In [ ]:
f1_score(start_logits, end_logits, start_positions, end_positions)

0.6002186955899961

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class GraphEnrichedRoberta(nn.Module):
  def __init__(self, model_name):
    super().__init__()
    self.llm_embedding_size = 768
    self.llm = AutoModelForQuestionAnswering.from_pretrained(model_name)

    for param in self.llm.roberta.encoder.parameters():
      param.requires_grad = False

  def forward(self, input_ids, attention_mask, start_positions, end_positions):
    return self.llm(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

In [ ]:
model = GraphEnrichedRoberta(model_name)

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>